# LLM - RAG (대규모 언어 모델 - 검색 증강 생성)

# 개요
이 데모는 GPT-4 모델을 검색 증강 생성(RAG) 설정에서 사용하는 방법을 보여줍니다. \
모델은 자연어 질문을 분석하고, 지식 기반에서 관련 정보를 검색한 후, 검색된 정보를 기반으로 답변을 제공합니다.



In [ ]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import tiktoken
from dotenv import load_dotenv
load_dotenv()

credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  azure_ad_token_provider=token_provider,
  api_version="2024-02-15-preview"
)

CHAT_COMPLETIONS_MODEL = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')

### 자연어 질문으로 시작하기

In [ ]:
input = "2025년 한국 대선에서 어느 대통령 후보가 당선이 되었나요?"

### 1단계: GPT-4o: 이 질문에 답하기 위해 무엇이 필요한가요?

In [8]:
system_prompt=f'''다음 질문에 가능한 한 최선을 다해 답하세요. 다음 도구에 액세스할 수 있습니다:

웹 검색: 웹을 사용하여 정보를 찾습니다.

아래 형식을 엄격하게 따라야 합니다:

질문: 반드시 답해야 하는 입력 질문
생각: 무엇을 해야 할지 항상 생각해야 합니다.
행동: 취해야 할 행동, [조사] 중 하나이어야 합니다.
행동 입력: 행동에 대한 입력

시작!
'''
user_prompt = f'''
Question: {input}
Thought:'''

In [ ]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":system_prompt},
                {"role":"user","content": user_prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)

질문: 2025년 한국 대선에서 어느 대통령 후보가 당선이 되었나요?
생각: 2025년 한국 대통령 선거 결과에 대한 최신 정보를 찾아야 합니다.
행동: [조사]
행동 입력: 2025년 한국 대통령 선거 결과


In [ ]:
,)

print(response.choices[0].message.content)

### 2단계: 더 많은 세부 정보를 위해 웹 검색


계속하기 전에 구독에서 검색 서비스를 생성하세요 [검색 리소스 생성](https://serpapi.com). \
이 샘플은 쿼리를 사용하여 웹 검색 API에 호출을 수행하고 관련 웹 검색 결과를 반환합니다.

In [11]:

import serpapi
import json

params = {
  "engine": "google",
  "q": "2025년 한국 대선에서 어느 대통령 후보가 당선될 가능성이 높은가요? 그 이유를 설명하세요.",
  "api_key": os.getenv("SERP_API_KEY"),
}
print()

response = serpapi.search(params)

# response 객체를 JSON 문자열로 변환 후, ensure_ascii=False로 한글이 깨지지 않게 출력
print(json.dumps(response.data, ensure_ascii=False, indent=2))

context = ''
organic_results = response["organic_results"]

for i, result in enumerate(organic_results):
    title = result.get("title")
    link = result.get("link")
    snippet = result.get("snippet")
    # Ensure the title is printed as a proper Korean string
    if isinstance(snippet, bytes):
        snippet = snippet.decode('utf-8')
    if isinstance(title, bytes):
        title = title.decode('utf-8')
    print(f"Result {i+1}:")
    print(f"Title: {title}")
    print(f"Link: {link}")
    print(f"Snippet:{snippet}")
    context += snippet + '\n'


{
  "search_metadata": {
    "id": "687a02a110b5cf2b26ebf982",
    "status": "Success",
    "json_endpoint": "https://serpapi.com/searches/d29be1be5bcaa72d/687a02a110b5cf2b26ebf982.json",
    "pixel_position_endpoint": "https://serpapi.com/searches/d29be1be5bcaa72d/687a02a110b5cf2b26ebf982.json_with_pixel_position",
    "created_at": "2025-07-18 08:15:29 UTC",
    "processed_at": "2025-07-18 08:15:29 UTC",
    "google_url": "https://www.google.com/search?q=2025%EB%85%84+%ED%95%9C%EA%B5%AD+%EB%8C%80%EC%84%A0%EC%97%90%EC%84%9C+%EC%96%B4%EB%8A%90+%EB%8C%80%ED%86%B5%EB%A0%B9+%ED%9B%84%EB%B3%B4%EA%B0%80+%EB%8B%B9%EC%84%A0%EB%90%A0+%EA%B0%80%EB%8A%A5%EC%84%B1%EC%9D%B4+%EB%86%92%EC%9D%80%EA%B0%80%EC%9A%94%3F+%EA%B7%B8+%EC%9D%B4%EC%9C%A0%EB%A5%BC+%EC%84%A4%EB%AA%85%ED%95%98%EC%84%B8%EC%9A%94.&oq=2025%EB%85%84+%ED%95%9C%EA%B5%AD+%EB%8C%80%EC%84%A0%EC%97%90%EC%84%9C+%EC%96%B4%EB%8A%90+%EB%8C%80%ED%86%B5%EB%A0%B9+%ED%9B%84%EB%B3%B4%EA%B0%80+%EB%8B%B9%EC%84%A0%EB%90%A0+%EA%B0%80%EB%8A%A5%EC%84%B1

### 추가 처리를 위한 수집된 정보

In [12]:
print(context)

이번 대선 후보들 가운데 여론조사 지지율 선두를 달리는 더불어민주당 이재명 후보와 국민의힘 김문수 두 후보가 밝힌 공약을 중심으로, 입법부, 행정부, ...
2025 대통령선거...당선 가를 마지막 변수는? 이현준 / KBS 기자 전원책 / 변호사 최창렬 / 용인대학교 특임교수 윤희웅 / 오피니언즈 대표 ...
출연 : 장성호 국민의힘 중앙선대위 공보부단장, 차재원 부산가톨릭대 특임교수] 이재명 더불어민주당 후보가 제21대 대통령선거에서 당선이 확실시 ...
출연: 서용주 전 민주당 상근부대변인·정광재 국민의힘 대변인] 역대 최다 득표로 당선된 이재명 대통령이 취임 첫날 국회에서 취임 선서를 하고 ...
박충구 교수는 계엄으로 인해 치러진 2025년 대선을 돌아보며, 기독교적 관점에서 진정한 통합은 민주주의 가치의 실현에 있으며, 이재명 정부가 국민 ...
진행 : 윤재희 앵커 □ 출연 : 김진욱 전 더불어민주당 대변인, 김철현 경일대 특임교수, 전용기 더불어민주당 의원, 정성국 국민의힘 의원 * 아래 ...
실제로 사전투표 첫째날에는 투표율이 20%에 근접하여 20대 대선 같은 날보다 2%p 상승하였으나, 둘째날에는 시간이 갈수록 투표율 상승폭이 둔화되어 ...
이번 한국의 조기 대통령 선거에서 보이는 특징 중 하나는 제1야당의 이재명 후보가 방탄조끼를 입고 유세를 펼친다는 점이다. 이 후보는 최근 한 ...
후보자는 자신의 정체성과 국민에게 이야기하는 메시지가 일관되어야 합니다. 이는 기업(브랜드)도 마찬가지이죠. 말이 계속 바뀌는 후보자에게 투표를 ...
출연 : 김동원 국민의힘 중앙선대위 대변인 · 조기연 민주당 법률위 부위원장] 제 21대 대통령 선거 투표가 모두 마무리되고 본격적인 개표 작업이 ...



In [13]:
system_prompt=f'''Answer the following questions as best you can. Use the context provided below:
{context}

Begin!
'''
user_prompt = f'''
Question: {input}\
Answer:'''

### 3단계: GPT-4o: 이 질문에 답하기 위해 무엇을 해야 하나요? (다시)

In [14]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":system_prompt},
                {"role":"user","content": user_prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)

2025년 한국 대통령선거에서 더불어민주당의 이재명 후보가 역대 최다 득표로 당선되었습니다.


### GPT-4o: 이제 질문에 대한 최종 답변을 얻었습니다.

#### 탐색할 사용 사례
1. **지식 관리** \
내부 문서, 매뉴얼 및 정책에서 정보를 검색하고 제공하여 기업 지식에 대한 접근성을 향상시킵니다.

2. **비즈니스 인텔리전스 및 분석** \
시장 분석가가 다양한 소스에서 관련 시장 데이터, 보고서 및 통찰력을 검색할 수 있도록 지원합니다.

3. **법률 및 준수** \
법률 전문가가 쿼리에 따라 관련 판례, 법령 및 법적 선례를 검색할 수 있도록 지원합니다.